<!-- NOTEBOOK_METADATA source: "⚠️ Jupyter Notebook" title: "Monitor Parallel AI Tasks with Langfuse" sidebarTitle: "Parallel AI" logo: "/images/integrations/parallel_icon.png" description: "Learn how to trace Parallel AI task execution using Langfuse to capture detailed observability data for your AI workflow operations." category: "Integrations" -->

# Parallel AI Integration

In this guide, we'll show you how to integrate [Langfuse](https://langfuse.com) with [Parallel AI](https://parallel.ai/) to trace your AI task operations. By leveraging Langfuse's tracing capabilities, you can automatically capture details such as inputs, outputs, and execution times of your Parallel AI tasks.

> **What is Parallel AI?** [Parallel AI](https://parallel.ai/) is an API service that enables you to execute AI tasks in parallel, optimizing workflow efficiency. It provides a powerful task API that allows you to run multiple AI operations concurrently, making it ideal for building scalable AI applications.

> **What is Langfuse?** [Langfuse](https://langfuse.com) is an open source LLM engineering platform that helps teams trace API calls, monitor performance, and debug issues in their AI applications.

## Get Started

First, install the necessary Python packages:

In [ ]:
%pip install langfuse requests

Next, configure your environment with your Langfuse API keys. You can obtain your keys from your Langfuse dashboard.

In [ ]:
import os
# Get keys for your project from the project settings page https://cloud.langfuse.com

os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_BASE_URL"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_BASE_URL"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

Set up your Parallel AI API key. You can obtain your API key from the [Parallel AI dashboard](https://parallel.ai/).

In [ ]:
PARALLEL_API_KEY = "your-parallel-api-key"

## Tracing Parallel AI Tasks

To monitor your Parallel AI tasks, we use the [Langfuse `@observe()` decorator](https://langfuse.com/docs/sdk/python/decorators). In this example, the `@observe()` decorator captures the inputs, outputs, and execution time of the functions that interact with Parallel AI. All trace data is automatically sent to Langfuse, allowing you to monitor your AI task operations in real time.

### Creating and Running a Single Task

In [ ]:
import requests
from langfuse import observe

@observe()
def create_parallel_task(prompt: str, api_key: str):
    """Create a task using Parallel AI's Task API."""
    url = "https://api.parallel.ai/v1/tasks"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "prompt": prompt,
        "model": "gpt-4"
    }
    
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()

@observe()
def get_task_result(task_id: str, api_key: str):
    """Retrieve the result of a task."""
    url = f"https://api.parallel.ai/v1/tasks/{task_id}"
    headers = {
        "Authorization": f"Bearer {api_key}"
    }
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# Example usage
task_result = create_parallel_task(
    "Explain the benefits of parallel processing in AI applications",
    PARALLEL_API_KEY
)
print(f"Task created with ID: {task_result.get('task_id')}")

### Running Multiple Tasks in Parallel

One of the key features of Parallel AI is the ability to run multiple tasks concurrently. Here's how to trace multiple parallel requests:

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

@observe()
def run_parallel_tasks(prompts: list[str], api_key: str):
    """Execute multiple AI tasks in parallel and monitor with Langfuse."""
    task_ids = []
    
    # Create all tasks
    with ThreadPoolExecutor(max_workers=len(prompts)) as executor:
        futures = [executor.submit(create_parallel_task, prompt, api_key) for prompt in prompts]
        
        for future in as_completed(futures):
            result = future.result()
            task_ids.append(result.get('task_id'))
    
    # Wait for tasks to complete and retrieve results
    results = []
    for task_id in task_ids:
        # Poll for completion (in production, use webhooks)
        time.sleep(2)
        result = get_task_result(task_id, api_key)
        results.append(result)
    
    return results

# Example: Run multiple tasks in parallel
prompts = [
    "What are the key benefits of using Langfuse for LLM observability?",
    "How does parallel processing improve AI application performance?",
    "Explain the concept of distributed tracing in AI systems."
]

results = run_parallel_tasks(prompts, PARALLEL_API_KEY)
print(f"Completed {len(results)} tasks")

## See Traces in Langfuse

After executing the traced functions, log in to your [Langfuse Dashboard](https://cloud.langfuse.com) to view detailed trace logs. You'll be able to see:

- Individual task creation and retrieval operations
- Parallel execution patterns and timing
- Input prompts and output results
- Performance metrics for each task

The nested trace structure will show how multiple tasks are executed in parallel, making it easy to identify bottlenecks and optimize your AI workflows.

### Example Trace View

When you run multiple tasks in parallel, Langfuse captures the entire workflow:

- The parent `run_parallel_tasks` span shows the overall execution time
- Child spans for each `create_parallel_task` call show individual task creation
- Additional child spans for `get_task_result` show result retrieval

This hierarchical view helps you understand the performance characteristics of your parallel AI operations and identify opportunities for optimization.

## Advanced: Custom Metadata and Attributes

You can enhance your traces with custom metadata to make debugging and analysis easier:

In [ ]:
from langfuse.decorators import langfuse_context

@observe()
def create_task_with_metadata(prompt: str, api_key: str, task_type: str = "general"):
    """Create a task with additional metadata for better observability."""
    
    # Add custom metadata to the trace
    langfuse_context.update_current_observation(
        metadata={
            "task_type": task_type,
            "prompt_length": len(prompt),
            "model": "gpt-4"
        }
    )
    
    url = "https://api.parallel.ai/v1/tasks"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "prompt": prompt,
        "model": "gpt-4"
    }
    
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    
    result = response.json()
    
    # Update observation with task ID
    langfuse_context.update_current_observation(
        metadata={"task_id": result.get('task_id')}
    )
    
    return result

# Example with metadata
task = create_task_with_metadata(
    "Summarize the benefits of observability in AI systems",
    PARALLEL_API_KEY,
    task_type="summarization"
)

<!-- MARKDOWN_COMPONENT name: "LearnMore" path: "@/components-mdx/integration-learn-more.mdx" -->